## Importing Libraries

In [1]:
import psycopg2
import pandas as pd

## Function to connect to Postgres

In [36]:
def create_db():
    conn = psycopg2.connect("host=pg dbname=postgres user=pguser password=asd123")
    print("Postgres connected!")
    cur = conn.cursor()
    print("Cursor created!")
    conn.set_session(autocommit=True)
    
    cur.execute("DROP DATABASE IF EXISTS accounts;")
    cur.execute("CREATE DATABASE accounts;")
    print("DB created!")
    
    conn.close()
    cur.close()
    
    conn = psycopg2.connect("host=pg dbname=accounts user=pguser password=asd123")
    print("Postgres connected to new db!")
    cur = conn.cursor()
    print("Cursor created!")
    conn.set_session(autocommit=True)
    
    return conn, cur

## Loading and Cleaning Data
### Account Country Data
#### Loading Data

In [6]:
AccountsCountry = pd.read_csv("data/AccountsCountry.csv")
AccountsCountry.head(3)

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,IBRD,NaN,Armenian dram,2020 (expected),"Demographic and Health Survey, 2015/16",NaN,...,"Integrated household survey (IHS), 2016",Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia,Armenia


#### Columns

In [8]:
AccountsCountry.columns

Index(['Code', 'Long Name', 'Income Group', 'Region', 'Lending category',
       'Other groups', 'Currency Unit', 'Latest population census',
       'Latest household survey', 'Special Notes',
       'National accounts base year', 'National accounts reference year',
       'System of National Accounts', 'SNA price valuation',
       'Alternative conversion factor', 'PPP survey years',
       'Balance of Payments Manual in use', 'External debt Reporting status',
       'System of trade', 'Government Accounting concept',
       'IMF data dissemination standard',
       'Source of most recent Income and expenditure data',
       'Vital registration complete', 'Latest agricultural census',
       'Latest industrial data', 'Latest trade data',
       'Latest water withdrawal data', '2-alpha code', 'WB-2 code',
       'Table Name', 'Short Name'],
      dtype='object')

#### Selecting Columns

In [9]:
AccountsCountry_clean = AccountsCountry[["Code", "Short Name", "Table Name", "Long Name", "Currency Unit"]]
AccountsCountry_clean.head(3)

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram


#### Removing NaN values

In [64]:
AccountsCountry_clean = AccountsCountry_clean.dropna()
AccountsCountry_clean.head(3)

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram


### Account Data
#### Loading Data

In [32]:
AccountData = pd.read_csv("data/AccountData.csv")
AccountData.head(3)

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3355.549,2868.997,3013.391,3259.893,3424.007,...,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3072.504,3178.417,3312.927


#### Columns

In [29]:
AccountData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

#### Renaming Columns

In [33]:
d = {
    "Series Name": "Indicator Name",
    "Series Code": "Indicator Code"
}
for name in AccountData.columns[4:]:
    d[name] = name[:4]

AccountData = AccountData.rename(columns=d)
AccountData.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1995,1996,1997,1998,1999,2000,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3355.549,2868.997,3013.391,3259.893,3424.007,...,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3072.504,3178.417,3312.927


#### Removing Columns

In [34]:
AccountData = AccountData.drop(AccountData.columns[4:-4], axis=1)
AccountData.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,2015,2016,2017,2018
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,2.987545e+03,3072.504,3178.417,3312.927


#### Replacing empty values

In [121]:
AccountData = AccountData.replace("..", 0)
AccountData.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,2015,2016,2017,2018
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,2.987545e+03,3072.504,3178.417,3312.927


### Account Series Data
#### Loading Data

In [18]:
AccountSeries = pd.read_csv("data/AccountSeries.csv")
AccountSeries.head(3)

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN


#### Columns

In [19]:
AccountSeries.columns

Index(['Code', 'Indicator Name', 'Long definition', 'Source', 'Topic',
       'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name'],
      dtype='object')

#### Selecting Columns

In [23]:
AccountSeries = AccountSeries[["Code", "Indicator Name", "Long definition", "Topic"]]
AccountSeries.head(3)

,Code,Indicator Name,Long definition,Topic
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,Human capital
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,Human capital
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,Human capital


## Connecting to DB

In [37]:
conn, cur = create_db()

Postgres connected!
Cursor created!
DB created!
Postgres connected to new db!
Cursor created!


## Creating Tables in DB
### Account Country Table

In [41]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS account_country (
        Code VARCHAR PRIMARY KEY,
        Short_Name VARCHAR,
        Table_Name VARCHAR,
        Long_Name VARCHAR,
        Currency_Unit VARCHAR
    );"""
)

### Account Data Table

In [42]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS account_data (
        Country_Name VARCHAR,
        Country_Code VARCHAR,
        Indicator_Name VARCHAR,
        Indicator_Code VARCHAR,
        y_2015 NUMERIC,
        y_2016 NUMERIC,
        y_2017 NUMERIC,
        y_2018 NUMERIC
    );"""
)

### Account Series Table

In [43]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS account_series (
        Code VARCHAR,
        Indicator_Name VARCHAR,
        Long_definition VARCHAR,
        Topic VARCHAR
    );"""
)

## Inserting values to tables
### Account Country Table

In [93]:
insert_country_values = """
    INSERT INTO account_country (
        Code,
        Short_Name,
        Table_Name,
        Long_Name,
        Currency_Unit
    ) VALUES """

values = ""
for _, row in AccountsCountry_clean.iterrows():
    clean_str = "~".join(list(row))
    clean_str = clean_str.replace("'", "")
    list_words_clean = clean_str.split("~")
    value = "('" + "','".join(list_words_clean) + "'),"
    values = values + value
values = values[:-1]

cur.execute(insert_country_values + values)

### Account Data Table

In [122]:
insert_data_values = """
    INSERT INTO account_data (
        Country_Name,
        Country_Code,
        Indicator_Name,
        Indicator_Code,
        y_2015,
        y_2016,
        y_2017,
        y_2018
    ) VALUES """

values = ""
for i, row in AccountData.iterrows():
    row = [str(e) for e in list(row)]
    clean_str = "~".join(list(row))
    clean_str = clean_str.replace("'", "")
    list_words_clean = clean_str.split("~")
    value = "('" + "','".join(list_words_clean) + "'),"
    values = values + value
values = values[:-1]

cur.execute(insert_data_values + values)

### Account Series Table

In [123]:
insert_series_values = """
    INSERT INTO account_series (
        Code,
        Indicator_Name,
        Long_definition,
        Topic
    ) VALUES """

values = ""
for i, row in AccountSeries.iterrows():
    row = [str(e) for e in list(row)]
    clean_str = "~".join(list(row))
    clean_str = clean_str.replace("'", "")
    list_words_clean = clean_str.split("~")
    value = "('" + "','".join(list_words_clean) + "'),"
    values = values + value
values = values[:-1]

cur.execute(insert_series_values + values)